In [99]:
import os
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from skimage.io import imread
from skimage.transform import resize
from os.path import join
from glob import glob

# Data import

In [100]:
# Load CSV file
gt = pd.read_csv('ground_truth.csv')
main_folder = 'tom_and_jerry'

In [101]:
gt

,filename,tom,jerry
0,frame0.jpg,0,0
1,frame1.jpg,0,0
2,frame2.jpg,0,0
3,frame3.jpg,0,0
4,frame4.jpg,0,0
...,...,...,...
5473,frame5473.jpg,0,0
5474,frame5474.jpg,0,0
5475,frame5475.jpg,0,0
5476,frame5476.jpg,0,0


# Functions

In [102]:
def prepare_data(images):
    X = []
    y = []

    for image in images:
        image_name = image.split('/')[2]
        image_read = imread(image)
        resized_image = resize(image_read, (150, 150,))
        flatten_image = resized_image.flatten()
        X.append(flatten_image)
        labels = np.array(gt.loc[gt['filename'] == image_name, ['tom', 'jerry']]).flatten()
        y.append(labels)

    return np.array(X), np.array(y)

# Data analysis

In [103]:
folders = ['jerry', 'tom', 'tom_jerry_0', 'tom_jerry_1']
for folder in folders:
    images = glob(join(main_folder, folder, '*.jpg'))
    print('Total amount of {}: {}'.format(folder, len(images)))

Total amount of jerry: 1240
Total amount of tom: 1930
Total amount of tom_jerry_0: 1528
Total amount of tom_jerry_1: 780


In [104]:
images = glob(join(main_folder, '*', '*.jpg'))
print('Total amount of images: {}'.format(len(images)))
print('List of images:', images)

Total amount of images: 5478
List of images: ['tom_and_jerry/tom/frame4966.jpg', 'tom_and_jerry/tom/frame583.jpg', 'tom_and_jerry/tom/frame4972.jpg', 'tom_and_jerry/tom/frame1822.jpg', 'tom_and_jerry/tom/frame2481.jpg', 'tom_and_jerry/tom/frame4796.jpg', 'tom_and_jerry/tom/frame1836.jpg', 'tom_and_jerry/tom/frame540.jpg', 'tom_and_jerry/tom/frame232.jpg', 'tom_and_jerry/tom/frame1177.jpg', 'tom_and_jerry/tom/frame3760.jpg', 'tom_and_jerry/tom/frame4769.jpg', 'tom_and_jerry/tom/frame4999.jpg', 'tom_and_jerry/tom/frame3748.jpg', 'tom_and_jerry/tom/frame4033.jpg', 'tom_and_jerry/tom/frame2442.jpg', 'tom_and_jerry/tom/frame3984.jpg', 'tom_and_jerry/tom/frame568.jpg', 'tom_and_jerry/tom/frame2324.jpg', 'tom_and_jerry/tom/frame2873.jpg', 'tom_and_jerry/tom/frame959.jpg', 'tom_and_jerry/tom/frame2867.jpg', 'tom_and_jerry/tom/frame4580.jpg', 'tom_and_jerry/tom/frame965.jpg', 'tom_and_jerry/tom/frame971.jpg', 'tom_and_jerry/tom/frame4594.jpg', 'tom_and_jerry/tom/frame1361.jpg', 'tom_and_jerry/t

In [105]:
print('Before shuffling: {}'.format(images[:10]))
np.random.shuffle(images)
print('shuffling: {}'.format(images[:10]))

Before shuffling: ['tom_and_jerry/tom/frame4966.jpg', 'tom_and_jerry/tom/frame583.jpg', 'tom_and_jerry/tom/frame4972.jpg', 'tom_and_jerry/tom/frame1822.jpg', 'tom_and_jerry/tom/frame2481.jpg', 'tom_and_jerry/tom/frame4796.jpg', 'tom_and_jerry/tom/frame1836.jpg', 'tom_and_jerry/tom/frame540.jpg', 'tom_and_jerry/tom/frame232.jpg', 'tom_and_jerry/tom/frame1177.jpg']
shuffling: ['tom_and_jerry/tom_jerry_1/frame4337.jpg', 'tom_and_jerry/tom/frame4922.jpg', 'tom_and_jerry/tom/frame4163.jpg', 'tom_and_jerry/tom_jerry_1/frame2269.jpg', 'tom_and_jerry/tom_jerry_0/frame1900.jpg', 'tom_and_jerry/tom/frame575.jpg', 'tom_and_jerry/tom_jerry_0/frame4646.jpg', 'tom_and_jerry/tom_jerry_1/frame5223.jpg', 'tom_and_jerry/tom/frame176.jpg', 'tom_and_jerry/tom/frame174.jpg']


In [106]:
print(len(images))
print(int(0.2*len(images)))


5478
1095


In [107]:
trainset = images[:int(0.6*len(images))]
valset = images[int(0.6*len(images)):int(0.8*len(images))]
testset = images[int(0.8*len(images)):]
print('Total: {} splitted in Train: {}, Val: {} and Test: {}'.format(len(images), len(trainset), len(valset), len(testset)))

Total: 5478 splitted in Train: 3286, Val: 1096 and Test: 1096


In [108]:
train_x, train_y = prepare_data(trainset)
val_x, val_y = prepare_data(valset)
test_x, test_y = prepare_data(testset)
print('Train: {}, Val: {} and Test: {}'.format(len(train_x), len(val_x), len(test_x)))
print('Total: {}'.format(len(train_x) + len(val_x) + len(test_x)))

Train: 3286, Val: 1096 and Test: 1096
Total: 5478


# Modelling - Support Vector Classifier

### Training

In [109]:
'''clf = SVC(kernel='rbf', decision_function_shape='ovr')
clf.fit(train_x, train_y)'''

"clf = SVC(kernel='rbf', decision_function_shape='ovr')\nclf.fit(train_x, train_y)"

# Modelling - Decision Tree Classifier

### Training

In [110]:
clf = DecisionTreeClassifier(random_state=42)
clf.fit(train_x, train_y)

DecisionTreeClassifier(random_state=42)

### Validation

In [111]:
print('Validation accuracy Decision Tree: {:.3f}'.format(clf.score(val_x, val_y)))

Validation accuracy Decision Tree: 0.564


### Test

In [112]:
pred_y = clf.predict(test_x)
print('Predicted {} samples: {}'.format(len(pred_y), pred_y))
print('GT {} samples: {}'.format(len(test_y), test_y))

Predicted 1096 samples: [[0 0]
 [0 1]
 [1 1]
 ...
 [0 1]
 [1 1]
 [1 1]]
GT 1096 samples: [[0 1]
 [0 0]
 [0 1]
 ...
 [0 1]
 [1 0]
 [1 1]]


### Accuracy

In [113]:
print('Final Accuracy Decision Tree: {:.3f}'.format(accuracy_score(test_y, pred_y)))

Final Accuracy Decision Tree: 0.587


In [114]:
# Classification report
pred_y = clf.predict(train_x)
print('---------------TRAIN SET---------------')
print(classification_report(train_y, pred_y))
pred_y = clf.predict(test_x)
print('---------------TEST SET---------------')
print(classification_report(test_y, pred_y))

---------------TRAIN SET---------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1616
           1       1.00      1.00      1.00      1222

   micro avg       1.00      1.00      1.00      2838
   macro avg       1.00      1.00      1.00      2838
weighted avg       1.00      1.00      1.00      2838
 samples avg       0.72      0.72      0.72      2838



/Users/ericsultini/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ericsultini/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


---------------TEST SET---------------
              precision    recall  f1-score   support

           0       0.76      0.77      0.77       563
           1       0.61      0.67      0.64       406

   micro avg       0.70      0.72      0.71       969
   macro avg       0.69      0.72      0.70       969
weighted avg       0.70      0.72      0.71       969
 samples avg       0.52      0.52      0.51       969



/Users/ericsultini/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ericsultini/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Modelling - Random Forest Classifier

### Training

In [115]:
clf = RandomForestClassifier(random_state=42, n_jobs=-1)
clf.fit(train_x, train_y)

RandomForestClassifier(n_jobs=-1, random_state=42)

### Validation

In [116]:
print('Validation accuracy RandomForest: {:.3f}'.format(clf.score(val_x, val_y)))

Validation accuracy RandomForest: 0.773


### Test

In [117]:
pred_y = clf.predict(test_x)
print('Predicted {} samples: {}'.format(len(pred_y), pred_y))
print('GT {} samples: {}'.format(len(test_y), test_y))

Predicted 1096 samples: [[0 1]
 [0 0]
 [0 0]
 ...
 [0 1]
 [1 0]
 [1 1]]
GT 1096 samples: [[0 1]
 [0 0]
 [0 1]
 ...
 [0 1]
 [1 0]
 [1 1]]


### Accuracy

In [118]:
print('Final Accuracy RandomForest: {:.3f}'.format(accuracy_score(test_y, pred_y)))

Final Accuracy RandomForest: 0.783


In [119]:
# Classification report
pred_y = clf.predict(train_x)
print('---------------TRAIN SET---------------')
print(classification_report(train_y, pred_y))
print('---------------TEST SET---------------')
print(classification_report(test_y, pred_y))

---------------TRAIN SET---------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1616
           1       1.00      1.00      1.00      1222

   micro avg       1.00      1.00      1.00      2838
   macro avg       1.00      1.00      1.00      2838
weighted avg       1.00      1.00      1.00      2838
 samples avg       0.72      0.72      0.72      2838

---------------TEST SET---------------


/Users/ericsultini/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ericsultini/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ValueError: Found input variables with inconsistent numbers of samples: [1096, 3286]